In [1]:
!nvidia-smi

Thu Sep 23 07:38:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 44 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595726 sha256=f1b1bc857c810e3a2a9254fb61eb367e00f699eb68e4bb57a6118d89a17625b8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 754 kB 5.4 MB/s 
     |████████████████████████████████| 3.0 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-bdi1u6z8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-bdi1u6z8
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=91229753e97dd357947a83ea64201dd66cfb4e5d6598890b004613f6c7bf5079
  Stored in directory: /tmp/pip-ephem-wheel-cache-lkrwpr1u/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
#from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [8]:
file_path = "./drive/MyDrive/210916_train.txt"
file_path2= "./drive/MyDrive/210923_eval.txt"

In [10]:
dataset_train = nlp.data.TSVDataset(file_path, field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(file_path2, field_indices=[1,2], num_discard_samples=1)

In [13]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [15]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/1270 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.097601294517517 train acc 0.328125
epoch 1 batch id 201 loss 0.674987256526947 train acc 0.627021144278607
epoch 1 batch id 401 loss 0.4131578207015991 train acc 0.6922147755610972
epoch 1 batch id 601 loss 0.6046237349510193 train acc 0.7156561980033278
epoch 1 batch id 801 loss 0.47075337171554565 train acc 0.7296933520599251
epoch 1 batch id 1001 loss 0.4261920154094696 train acc 0.7396197552447552
epoch 1 batch id 1201 loss 0.43728604912757874 train acc 0.7476582014987511
epoch 1 train acc 0.7498267716535433


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 test acc 1.0


...

epoch 4 test acc 1.0


  0%|          | 0/1270 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.20840556919574738 train acc 0.90625
epoch 5 batch id 201 loss 0.2425011396408081 train acc 0.8935012437810945
epoch 5 batch id 401 loss 0.2035074234008789 train acc 0.8993142144638404
epoch 5 batch id 601 loss 0.2758948802947998 train acc 0.902740224625624
epoch 5 batch id 801 loss 0.12938502430915833 train acc 0.9057428214731585
epoch 5 batch id 1001 loss 0.21806325018405914 train acc 0.9076548451548452
epoch 5 batch id 1201 loss 0.1830545961856842 train acc 0.9098017277268943
epoch 5 train acc 0.9100556102362206


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 5 test acc 1.0


In [ ]:
import sys
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
!ls ./drive/MyDrive

'제목 없는 문서 (1).gdoc'
'스크린샷, 2019-11-29 1.zip'
 맥도날드_2021-09-01_2021-09-12.csv
 210916_test.txt
 210916_train221312.txt
 210916_train2.txt
 210916_train3333.txt
 210916_train.txt
 광주_2반_최광진.jpg
 광주캠퍼스_2반_최광진_현대오토에버.pdf
'광주_3반_최광진(0511679).pdf'
'광주_3반_최광진 (1).jpg'
'광주_3반_최광진 (2).jpg'
'광주_3반_최광진 (3).jpg'
'광주_3반_최광진 (4).jpg'
 광주_3반_최광진_자소서.docx
 광주_3반_최광진_입사지원서.docx
 광주_3반_최광진.docx
 광주_3반_최광진.jpg
 싸움_annotation_작업.pptx
 Badge.zip
 barcode.gdoc
 bigdata.zip
'bookmarks_20. 12. 16..html'
 Celeste.kor.patch.v1.02.by.LSW.zip
 colab
'Colab Notebooks'
 data.json
 default.png
'채용박람회 자기소개서.docx'
 d-tux
 foo.txt
 game
'제목 없는 문서.gdoc'
 gg01.jpg
'제목 없는 프레젠테이션.gslides'
 hadoop
'ICT Internship 소프트온넷(주)'
'ICT인턴십 작업사진.zip'
'Instrument List [4호 보일러 환경설비(SCR) 구축].xlsx'
 kmooc_HW

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/model1st.pt')

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/model1st.pth')

In [22]:
#기존 모델 불러와서 테스트 데이터 학습
model.load_state_dict(torch.load('./drive/MyDrive/model1st.pth'))

<All keys matched successfully>

In [28]:
train_acc = 0.0
test_acc = 0.0

model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(1, test_acc / (batch_id+1)))

  0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 test acc 0.8248906705539358
